In [1]:
import pandas as pd

from task2.services.converters.converter import convert_to_employees
from task2.services.db.data_inserter import insert_data
from task2.services.schema.schema_validation import SchemaValidation
from task2.services.transformations.employees_transformations import transform_csv_data
from task2.utls.logger_config import logger
from task2.models.base import Base, engine
from task2.services.schema.schema_definitions import EMPLOYEE_SCHEMA

# Create all tables in the database
Base.metadata.create_all(engine)
print("Database tables created successfully!")

2025-04-13 02:29:36,934 INFO sqlalchemy.engine.Engine BEGIN (implicit)
02:29:36 INFO: BEGIN (implicit)
2025-04-13 02:29:36,934 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("departments")
02:29:36 INFO: PRAGMA main.table_info("departments")
2025-04-13 02:29:36,935 INFO sqlalchemy.engine.Engine [raw sql] ()
02:29:36 INFO: [raw sql] ()
2025-04-13 02:29:36,936 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("departments")
02:29:36 INFO: PRAGMA temp.table_info("departments")
2025-04-13 02:29:36,936 INFO sqlalchemy.engine.Engine [raw sql] ()
02:29:36 INFO: [raw sql] ()
2025-04-13 02:29:36,937 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("employees")
02:29:36 INFO: PRAGMA main.table_info("employees")
2025-04-13 02:29:36,938 INFO sqlalchemy.engine.Engine [raw sql] ()
02:29:36 INFO: [raw sql] ()
2025-04-13 02:29:36,938 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("employees")
02:29:36 INFO: PRAGMA temp.table_info("employees")
2025-04-13 02:29:36,939 INFO sqlalchemy.

In [2]:
logger.info("Starting transformation: first trim, then convert types\n")
emp_df:pd.DataFrame =  pd.read_csv("./data/employees.csv", delimiter=',', header=0)

# Apply the transformation: first trim, then convert types.
transformed_emp_df = transform_csv_data(emp_df, EMPLOYEE_SCHEMA)
print(transformed_emp_df.head(1))

02:29:36 INFO: Starting transformation: first trim, then convert types

trim_all_values executed in 0.0009 seconds
convert_data_types executed in 0.0015 seconds
                                     id       name date_of_birth  salary  \
0  1dd3b13a-0b76-4468-b7c0-39ef57828315  James Lee    1973-09-15  145198   

   department_id  
0              8  


In [3]:
# Validate Schema
schema_validation = SchemaValidation(transformed_emp_df)
type_check = schema_validation.validate_data_types(type_schema=EMPLOYEE_SCHEMA)

In [4]:
# Convert DataFrame to Employees ORM instances
employee_instances = convert_to_employees(transformed_emp_df)

# Use the inserter to insert employee data into the database
insert_data(engine, employee_instances)

2025-04-13 02:29:37,107 INFO sqlalchemy.engine.Engine BEGIN (implicit)
02:29:37 INFO: BEGIN (implicit)
2025-04-13 02:29:37,109 INFO sqlalchemy.engine.Engine INSERT INTO employees (name, date_of_birth, salary, department_id, created_at, updated_at) VALUES (?, ?, ?, ?, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP) RETURNING id, created_at, updated_at
02:29:37 INFO: INSERT INTO employees (name, date_of_birth, salary, department_id, created_at, updated_at) VALUES (?, ?, ?, ?, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP) RETURNING id, created_at, updated_at
2025-04-13 02:29:37,110 INFO sqlalchemy.engine.Engine [generated in 0.00028s (insertmanyvalues) 1/100 (ordered; batch not supported)] ('James Lee', '1973-09-15', 145198, 8)
02:29:37 INFO: [generated in 0.00028s (insertmanyvalues) 1/100 (ordered; batch not supported)] ('James Lee', '1973-09-15', 145198, 8)
2025-04-13 02:29:37,111 INFO sqlalchemy.engine.Engine INSERT INTO employees (name, date_of_birth, salary, department_id, created_at, updated_at) VALU